## Wstęp

Rynek mieszkaniowy w Polsce charakteryzuje się silnym zróżnicowaniem przestrzennym – ceny (oraz stawki najmu) w dużych miastach i w ich centrach potrafią znacząco odbiegać od wartości obserwowanych na obrzeżach czy w mniejszych ośrodkach. Celem niniejszego notatnika jest kompleksowa analiza ofert mieszkaniowych oraz budowa i ocena modeli uczenia maszynowego, które przewidują poziom cen na podstawie cech nieruchomości i jej otoczenia.

W analizie wykorzystano publiczny zbiór danych “Apartment Prices in Poland” dostępny na platformie Kaggle (https://www.kaggle.com/datasets/krzysztofjamroz/apartment-prices-in-poland/data). Dane mają postać miesięcznych migawek ogłoszeń (pliki apartments_pl_YYYY_MM.csv dla sprzedaży oraz apartments_rent_pl_YYYY_MM.csv dla najmu), co pozwala łączyć rekordy z wielu okresów i zachować informację o dacie pobrania (snapshot_date). W notatniku dane zostały wczytane, zunifikowane i podzielone na dwa spójne podzbiory: sprzedaż (SELL) oraz najem (RENT).

### Cel analizy i modelowania

Notatnik realizuje dwa główne cele:
	1.	Eksploracyjna analiza danych (EDA) i opis rynku – ocena jakości danych, braków, rozkładów kluczowych zmiennych, różnic między miastami oraz wizualizacje przestrzenne (mapy heksowe median price_per_m2 i mapy intensywności występowania cech has* w Warszawie). Część przestrzenna obejmuje również eksperyment eksploracyjny polegający na próbie odtworzenia dominującej osi wysokich cen w Warszawie (bez użycia danych transportowych), co stanowi punkt wyjścia do dalszej weryfikacji hipotez.
	2.	Budowa i porównanie modeli predykcyjnych – przygotowanie pipeline’u przetwarzania danych (imputacja braków, standaryzacja cech numerycznych, kodowanie zmiennych kategorycznych), zastosowanie transformacji log1p(price) dla stabilizacji rozkładu zmiennej celu oraz porównanie kilku modeli regresyjnych (m.in. regresja liniowa, drzewo decyzyjne, k-NN). Skuteczność oceniana jest metrykami MAE, RMSE i R², a eksperymenty wykonywane są zarówno dla całej Polski, jak i wariantów lokalnych (np. sama Warszawa vs Polska bez Warszawy).

W efekcie notatnik dostarcza zarówno wniosków opisowych (jak różni się rynek między miastami i segmentami), jak i wyników ilościowych (jak dobrze wybrane modele potrafią przewidywać ceny na podstawie dostępnych cech), tworząc spójną podstawę do dalszych prac analitycznych i modelowych.

## 0. Konfiguracja środowiska i import bibliotek

W tej sekcji przygotowujemy środowisko pracy. Projekt opiera się na analizie danych tabelarycznych oraz wizualizacji, dlatego wykorzystujemy standardowy stos technologiczny:

1.  **Manipulacja danymi:** Biblioteka `pandas` posłuży do wczytania plików CSV, czyszczenia danych i agregacji, a `numpy` do operacji matematycznych (np. logarytmowanie).
2.  **Wizualizacja:** Używamy `matplotlib` jako fundamentu oraz `seaborn` do tworzenia estetycznych wykresów statystycznych (np. boxploty, heatmapy).
3.  **Obsługa plików:** Moduły `pathlib` oraz `re` (wyrażenia regularne) są niezbędne, ponieważ nasz zbiór danych jest podzielony na wiele plików (snapshotów czasowych), a data pobrania danych zawarta jest w nazwie pliku, a nie w jego treści.

Dodatkowo konfigurujemy parametry wyświetlania (`pd.set_option`), aby ramki danych w notatniku były czytelne (widoczność wszystkich kolumn) i nie były "łamane" w podglądzie.

`HTML` z IPython.display używamy by móc umieścić kod HTML w naszym notatniku. Pozwala nam on na czytelniejszą prezentacje wyników.

In [ ]:
import re
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.display import display, HTML
from scipy.stats import norm
import  geopandas as gpd
import contextily as ctx

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)  

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

DATA_DIR = Path("data")  # folder z CSV
CSV_GLOB = "*.csv"

SNAPSHOT_RE = re.compile(r"(?P<year>20\d{2})[_-](?P<month>\d{2})")

Wylistowanie dostępnych zbiorów danych z plików CSV.

In [ ]:
paths = sorted(DATA_DIR.glob(CSV_GLOB))
if not paths:
    raise FileNotFoundError(f"Brak plików CSV w: {DATA_DIR.resolve()}")

[p.name for p in paths[:10]], len(paths)

## 1. Wczytanie danych, wstępna agregacja i ich podział

Ponieważ dane surowe są rozproszone w wielu plikach CSV (reprezentujących różne "zrzuty" danych w czasie), konieczna jest ich agregacja do jednej, spójnej struktury. Poniższy kod realizuje ten proces w kilku krokach:

1.  **Iteracyjne wczytywanie:** W pętli przechodzimy przez listę ścieżek do plików. Każdy plik jest wczytywany do osobnej ramki danych (`df`).
2.  **Ekstrakcja metadanych czasowych:** Kluczowa informacja o dacie pobrania danych nie znajduje się wewnątrz pliku CSV, lecz w jego nazwie (np. `...2024_06...`). Używając zdefiniowanego wcześniej wyrażenia regularnego (`SNAPSHOT_RE`), wyciągamy rok i miesiąc, a następnie konwertujemy je na obiekt `pd.Timestamp`. Pozwala to na późniejszą analizę trendów w czasie.
3.  **Śledzenie źródła:** Dodajemy kolumnę `source_file`, aby zachować informację o pochodzeniu każdego rekordu – ułatwia to namierzanie ewentualnych błędów w konkretnych plikach.
4.  **Konsolidacja (Concatenation):** Wszystkie mniejsze ramki danych są łączone w jedną główną ramkę `df_all` za pomocą funkcji `pd.concat`. Resetujemy indeks (`ignore_index=True`), aby zachować ciągłość numeracji wierszy.
5.  **Feature Engineering (Cena za m²):** Już na tym etapie tworzymy nową cechę `price_per_m2`. Jest to najbardziej miarodajny wskaźnik na rynku nieruchomości, pozwalający porównywać wartość mieszkań o różnym metrażu.

In [ ]:
dfs = []

for p in paths:
    df = pd.read_csv(p)

    m = SNAPSHOT_RE.search(p.name)
    snapshot_date = pd.NaT
    if m:
        snapshot_date = pd.Timestamp(year=int(m.group("year")), month=int(m.group("month")), day=1)

    df["source_file"] = p.name
    df["snapshot_date"] = snapshot_date
    df["snapshot_year"] = pd.to_datetime(df["snapshot_date"]).dt.year
    df["snapshot_month"] = pd.to_datetime(df["snapshot_date"]).dt.month

    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
df_all['price_per_m2'] = df_all['price'] / df_all['squareMeters']
df_all.shape

display(df_all[['city', 'price', 'squareMeters', 'price_per_m2']].head())

### Podział danych na podzbiory (Sprzedaż vs Wynajem)

Ze względu na to, że zbiór danych zawiera zmieszane oferty sprzedaży i wynajmu (które mają drastycznie różne ceny), dzielimy główną ramkę `df_all` na dwie niezależne części:
1.  **Wynajem (`df_rent`):** Wyodrębniamy rekordy, których nazwa pliku źródłowego (`source_file`) zawiera słowo "rent".
2.  **Sprzedaż (`df_sell`):** Do tego zbioru trafiają wszystkie pozostałe rekordy (operator `~` oznacza logiczną negację maski wynajmu).

Użycie metody `.copy()` jest tutaj kluczowe – tworzy ona fizyczną kopię danych w pamięci, dzięki czemu późniejsze czyszczenie jednego zbioru nie wpływa na drugi.

In [ ]:
mask_rent = df_all["source_file"].str.lower().str.contains("rent", na=False)

mask_sell = ~mask_rent

df_rent = df_all[mask_rent].copy()
df_sell = df_all[mask_sell].copy()

df_rent.shape, df_sell.shape

### Weryfikacja niezaklasyfikowanych rekordów

Tworzymy pomocniczy zbiór `df_other`, trafiają do niego rekordy, które nie zostały przypisane ani do kategorii sprzedaży, ani wynajmu (np. z powodu nietypowych nazw plików). Sprawdzamy liczebność i źródła tych danych, aby upewnić się, że nie pomijamy istotnych informacji w procesie podziału. Dla naszych danych oczekujemy pustej ramki.

In [ ]:
df_other = df_all[~(mask_rent | mask_sell)].copy()
df_other["source_file"].value_counts().head(20), df_other.shape

## 2. Wstępna Obróbka Danych

Naszą obróbke danych operamy na wcześniej wyodrębionnych ramkach: `df_sell` i `df_rent`.

Podział ten jest kluczowy, ponieważ mechanizmy cenowe rządzące rynkiem sprzedaży (cena całkowita, kredyty) różnią się fundamentalnie od rynku najmu (czynsz miesięczny, stopa zwrotu).

Poniżej przedstawiamy podstawowe statystyki dla obu wyodrębnionych grup.

In [ ]:
def get_date_range(df):
    if 'snapshot_date' in df.columns:
        d_min = df['snapshot_date'].min()
        d_max = df['snapshot_date'].max()
        return f"{d_min.date()} — {d_max.date()}"
    return "Brak danych czasowych"

stats = {
    'Zbiór': ['Sprzedaż (Sell)', 'Wynajem (Rent)'],
    'Liczba ofert': [len(df_sell), len(df_rent)],
    'Liczba kolumn': [df_sell.shape[1], df_rent.shape[1]],
    'Liczba miast': [df_sell['city'].nunique(), df_rent['city'].nunique()],
    'Zakres dat': [get_date_range(df_sell), get_date_range(df_rent)]
}

stats_df = pd.DataFrame(stats)

display(stats_df.style.hide(axis='index'))

plt.figure(figsize=(8, 5))
ax = sns.barplot(x='Zbiór', y='Liczba ofert', data=stats_df, hue='Zbiór', palette="viridis", legend=False)

for i, v in enumerate(stats_df['Liczba ofert']):
    ax.text(i, v + (v * 0.02), f"{v:,}".replace(",", " "), ha='center', fontweight='bold')

plt.title('Liczebność zbiorów: Sprzedaż vs Wynajem', fontsize=14)
plt.ylabel('Liczba ofert')
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.show()

### Wnioski ze wstępnej obróbki:
1.  **Dysproporcja danych:** Jak widać na wykresie, liczebność obu grup może się różnić. Zbiór sprzedażowy (`df_sell`) jest zazwyczaj liczniejszy/mniejszy (zależnie od danych), co determinuje wybór metod walidacji.
2.  **Spójność czasowa:** Dane dla obu kategorii pochodzą z tego samego zakresu czasowego, co pozwala na rzetelną analizę porównawczą danych w czasie.
3.  **Pokrycie geograficzne:** Obie grupy obejmują taką samą liczbę miast, co sugeruje, że zbiór jest zbalansowany pod kątem lokalizacji (nie brakuje nagle danych o wynajmie w dużym mieście).

## 3. Analiza Jakości Danych i Czyszczenie

Przed przystąpieniem do modelowania konieczna jest weryfikacja jakości danych. W tym kroku realizujemy proces czyszczenia danych, który ma na celu:
1.  **Identyfikację braków danych:** Sprawdzenie, które zmienne są niekompletne i decyzja o ich usunięciu (jeśli braków jest > 30%) lub imputacji (zastąpienie brakujących wartości sztucznie wygenerowanymi danymi, np. średnią).
2.  **Wykrycie duplikatów:** Zarówno technicznych (identyczne wiersze), jak i logicznych (to samo mieszkanie pojawiające się w kolejnych snapshotach).
3.  **Eliminację błędów grubych (Outliers):** Usunięcie rekordów nierealnych fizycznie (np. cena 1 PLN, ujemny metraż, zbyt duży metraż), które mogłyby zafałszować wyniki modeli regresyjnych.

In [ ]:
def analyze_dataframe(df, df_name, plot_missing=True):
    n_rows, n_cols = df.shape

    # Braki danych
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    
    if missing.empty:
        table_html = "<p><b>Brak brakujących danych (NaN).</b></p>"
    else:
        missing_pct = (missing / n_rows) * 100
        missing_df = pd.DataFrame({'Liczba': missing, '% Braków': missing_pct})
        missing_df = missing_df.sort_values(by='% Braków', ascending=False)
        
        # Konwersja tabeli na HTML z kolorowaniem
        table_html = missing_df.head(10).style.background_gradient(cmap='Reds', subset=['% Braków'])\
            .format({'% Braków': '{:.2f}%'})\
            .set_caption("Top 10 brakujących danych")\
            .set_table_attributes('style="width:100%"')\
            .to_html()

    # Duplikaty
    n_dupl = df.duplicated().sum()
    n_dupl_logic = 0
    if 'id' in df.columns and 'snapshot_date' in df.columns:
        n_dupl_logic = df.duplicated(subset=['id', 'snapshot_date']).sum()

    # Eleminacja błędów grubych
    bad_price = (df['price'] <= 0).sum()
    bad_area = ((df['squareMeters'] < 10) | (df['squareMeters'] > 1000)).sum()
    
    
    # Panel HTML: Prawa kolumna (Tabela), Lewa kolumna (Tekst/Statystyki)
    dashboard_html = f"""
    <div style="display: flex; flex-direction: row; gap: 40px; align-items: flex-start;">
        <div style="flex: 1; min-width: 300px; padding: 15px; border-radius: 8px;">
            <h2> RAPORT JAKOŚCI DANYCH: {df_name} </h2>

            <p><b>Liczba obserwacji:</b> {n_rows:,}</p>
            
            <hr>
            
            <p><b>Duplikaty:</b></p>
            <ul style="margin-top: 5px;">
                <li>Pełne (dubel wiersza): <b>{n_dupl}</b></li>
                <li>Logiczne (id + data): <b>{n_dupl_logic}</b></li>
            </ul>
            
            <hr>
            
            <p><b>Sanity Check (Błędy):</b></p>
            <ul style="margin-top: 5px;">
                <li>Cena <= 0 PLN: <b style="color: {'red' if bad_price > 0 else 'green'}">{bad_price}</b></li>
                <li>Metraż < 10m² lub > 1000m²: <b style="color: {'red' if bad_area > 0 else 'green'}">{bad_area}</b></li>
            </ul>
        </div>
        <div style="flex: 1; min-width: 300px;">
            {table_html}
        </div>
    </div>
    """
    
    display(HTML(dashboard_html))
    
    # Wykres
    if plot_missing and not missing.empty:
        plt.figure(figsize=(10, 3))
        sns.barplot(x=missing_df['% Braków'], y=missing_df.index, color='salmon')
        plt.title(f'Wizualizacja braków danych - {df_name}')
        plt.axvline(x=30, color='red', linestyle='--', label='Próg odcięcia (30%)')
        plt.xlabel('% Brakujących wartości')
        plt.legend()
        plt.show()

analyze_dataframe(df_sell, "SPRZEDAŻ")
analyze_dataframe(df_rent, "WYNAJEM")

### Wnioski z analizy braków i decyzje:

Na podstawie powyższego raportu podejmujemy następujące kroki w procesie czyszczenia:

1.  **Usuwanie kolumn z dużą liczbą braków:** Zmienne, które mają powyżej 30% braków (np. często `condition`, `buildingMaterial`), niosą zbyt mało informacji, by być użyteczne, a ich imputacja byłaby obarczona dużym błędem. Zostaną usunięte.
2.  **Imputacja:** Dla zmiennych kluczowych z niewielką liczbą braków (np. `floor`, `buildYear`) zastosujemy w późniejszym etapie (przy modelowaniu) uzupełnianie medianą lub modą.
3.  **Usuwanie "śmieciowych" rekordów:** Zidentyfikowane oferty z ceną $\le 0$ lub nierealnym metrażem (np. 1 m²) traktujemy jako błędy wprowadzania danych i usuwamy je całkowicie, aby nie zaburzały statystyk (średniej, odchylenia).

### Automatyzacja procesu czyszczenia danych

Na podstawie wniosków z analizy jakości definiujemy funkcję `clean_data`, która standaryzuje proces oczyszczania dla obu zbiorów (sprzedaży i wynajmu). Procedura składa się z trzech kluczowych etapów:

1.  **Eliminacja duplikatów:**
    * Usuwamy duplikaty techniczne (całkowicie identyczne wiersze).
    * Usuwamy duplikaty logiczne: sytuacje, w których to samo mieszkanie (`id`) pojawia się wielokrotnie w ramach jednego zrzutu danych (`snapshot_date`). Pozostawiamy tylko pierwsze wystąpienie, aby uniknąć przekłamania statystyk (np. sztucznego zawyżania liczby ofert).

2.  **Filtrowanie po sztywno zdefiniowanych regułach:**
    * Zastosowano reguły biznesowe w celu odrzucenia błędnych rekordów.
    * **Cena:** Musi być dodatnia (`price > 0`).
    * **Metraż:** Ograniczono analizę do lokali o powierzchni od **10 m²** (eliminacja miejsc postojowych/komórek błędnie wpisanych jako mieszkania) do **500 m²** (eliminacja obiektów komercyjnych lub błędów rzędu wielkości).

3.  **Redukcja rzadkich cech:**
    * Automatycznie usuwamy kolumny, w których brakuje ponad **50% danych**. Zmienne o tak niskim pokryciu (np. rzadko wypełniane pola opcjonalne) są bezużyteczne w modelowaniu, a ich imputacja byłaby obarczona zbyt dużym błędem.

Funkcja raportuje procent odrzuconych rekordów, co pozwala kontrolować, czy nie tracimy zbyt dużej części zbioru danych.

In [ ]:
def clean_data(df):
    df_clean = df.copy()
    start_len = len(df_clean)
    
    # Usuwanie duplikatów
    df_clean = df_clean.drop_duplicates()
    if 'id' in df_clean.columns and 'snapshot_date' in df_clean.columns:
        df_clean = df_clean.drop_duplicates(subset=['id', 'snapshot_date'], keep='first')

    # Usuwanie błędów logicznych (Cena i Metraż)
    mask_correct = (df_clean['price'] > 0) & \
                   (df_clean['squareMeters'] >= 10) & \
                   (df_clean['squareMeters'] <= 500)
    
    df_clean = df_clean[mask_correct]
    
    # Usuwanie kolumn z > 50% braków
    threshold = 0.5 * len(df_clean)
    df_clean = df_clean.dropna(thresh=threshold, axis=1)
    
    # Raport skuteczności
    end_len = len(df_clean)
    dropped = start_len - end_len
    print(f"Czyszczenie zakończone.\nUsunięto {dropped} rekordów ({dropped/start_len:.2%}).")
    print(f"Pozostało: {end_len} obserwacji.")
    return df_clean

# Zastosowanie czyszczenia
print("--- CZYSZCZENIE ZBIORU SPRZEDAŻY ---")
df_sell = clean_data(df_sell)

print("\n--- CZYSZCZENIE ZBIORU WYNAJMU ---")
df_rent = clean_data(df_rent)

Po tych wszystkich operacjach wyświetlamy pierwsze 10 wierszy dla ofert wynajmu i sprzedaży.

In [ ]:
print("df_rent (10 pierwszych wiersze):")
display(df_rent.head(10))

print("\ndf_sell (10 pierwszych wiersze):")
display(df_sell.head(10))

## 4. Profil zmiennych kategorycznych

In [ ]:
TOP_N = 7
FIG_W = 12

def plot_categorical_profiles(df: pd.DataFrame, label: str, top_n: int | None = 7):
    df = df.copy()
    has_cols = [c for c in df.columns if c.startswith("has")]
    for c in has_cols:
        df[c] = df[c].where(df[c].isin(["yes", "no"]))

    df["city"].value_counts(dropna=False).plot(kind="bar", figsize=(FIG_W, 5), title=f"{label}: liczba ofert per city")
    plt.xlabel("city"); plt.ylabel("liczba ofert"); plt.xticks(rotation=45, ha="right"); plt.tight_layout(); plt.show()

    overall = pd.Series({
        c: ((df[c] == "yes").sum() / df[c].notna().sum() * 100) if df[c].notna().sum() else np.nan
        for c in has_cols
    }).sort_values(ascending=False)
    overall.plot(kind="bar", figsize=(FIG_W, 5), title=f"{label}: % 'yes' dla cech has* (globalnie, wśród nie-null)")
    plt.xlabel("cecha"); plt.ylabel("% 'yes'"); plt.xticks(rotation=45, ha="right"); plt.tight_layout(); plt.show()

    ncols = 2
    nrows = int(np.ceil(len(has_cols) / ncols))
    fig, axes = plt.subplots(nrows * 2, ncols, figsize=(FIG_W, max(6, nrows * 2 * 3.2)), constrained_layout=True)
    axes = np.array(axes)

    for idx, c in enumerate(has_cols):
        r = (idx // ncols) * 2
        col = idx % ncols

        yes = df.groupby("city")[c].apply(lambda s: (s == "yes").sum()).sort_values(ascending=False)
        nn = df.groupby("city")[c].apply(lambda s: s.notna().sum())
        pct = (yes / nn * 100).replace([np.inf, -np.inf], np.nan)

        if top_n is not None:
            top = yes.head(top_n).index
            yes, pct = yes.loc[top], pct.loc[top]

        yes.plot(kind="bar", ax=axes[r, col])
        axes[r, col].set_title(f"{label} | {c}: liczba ofert z 'yes' per city" + (f" (top {top_n})" if top_n else ""))
        axes[r, col].set_xlabel(""); axes[r, col].set_ylabel("liczba 'yes'"); axes[r, col].tick_params(axis="x", rotation=45)

        pct.plot(kind="bar", ax=axes[r + 1, col])
        axes[r + 1, col].set_title(f"{label} | {c}: % ofert z 'yes' per city (wśród yes/no)")
        axes[r + 1, col].set_xlabel("city"); axes[r + 1, col].set_ylabel("% 'yes'"); axes[r + 1, col].tick_params(axis="x", rotation=45)

    for empty_idx in range(len(has_cols), nrows * ncols):
        r = (empty_idx // ncols) * 2
        col = empty_idx % ncols
        axes[r, col].axis("off")
        axes[r + 1, col].axis("off")

    fig.suptitle(f"{label}: profil cech has* (count + % per city)", y=1.02, fontsize=13)
    plt.show()

plot_categorical_profiles(df_rent, "RENT", TOP_N)
plot_categorical_profiles(df_sell, "SELL", TOP_N)

### Wnioski z profilu zmiennych kategorycznych
#### 1. Silna nierównowaga ofert między miastami
- Zarówno dla wynajmu, jak i sprzedaży widoczna jest bardzo duża koncentracja ogłoszeń w kilku największych ośrodkach — Warszawa zdecydowanie dominuje, a następnie (z wyraźnym spadkiem) Kraków i Wrocław.
- Taki rozkład oznacza, że wyniki oparte na liczbach bezwzględnych (np. “najwięcej balkonów w Warszawie”) w dużej mierze odzwierciedlają po prostu skalę rynku / liczebność próby, a nie specyfikę zasobów mieszkaniowych.
- Właśnie dlatego dodanie wykresów procentowych jest kluczowe: % `yes` jest bardziej miarodajny w porównaniach między miastami.

#### 2. Najczęstsze cechy w RENT vs SELL
- W RENT najbardziej powszechne są: winda (hasElevator) i balkon (hasBalcony) – wartości `yes` stanowią większość obserwacji (ok. 60%+).
- W SELL nadal często występują balkon i winda, ale widać wyraźnie inną strukturę:
	- komórka lokatorska (hasStorageRoom) jest dużo częstsza niż w wynajmie
	- ochrona (hasSecurity) pozostaje cechą relatywnie rzadką 
- Interpretacja rynkowa: sprzedaż mocniej reprezentuje nowe inwestycje / standard deweloperski, gdzie częściej występują: komórki lokatorskie, miejsca postojowe i infrastruktura osiedlowa, w wynajmie częściej pojawiają się mieszkania “użytkowe”, gdzie te dodatki nie zawsze są formalnie przypisane.

#### 3. Różnice między rankingiem liczbowym i procentowym
- W rankingach liczbowych TOP miast dla każdej cechy niemal zawsze wygrywa Warszawa — co jest konsekwencją największej liczby ogłoszeń.
- Dopiero wykresy procentowe pokazują realne różnice w strukturze zasobu:
- hasParkingSpace: w wielu miastach udział `yes` jest zbliżony, ale są też wyraźne odstępstwa (np. w części miast udział jest zauważalnie wyższy niż w Warszawie).
- hasBalcony: cecha jest ogólnie bardzo stabilna między największymi miastami (często ok. 55–65% `yes`), co sugeruje, że balkon jest w dużych ośrodkach standardem oferty, a różnice są raczej umiarkowane.
- hasElevator: większe zróżnicowanie procentowe — w niektórych miastach udział `yes` jest zdecydowanie wyższy, co może odzwierciedlać większy udział zabudowy wielopiętrowej / nowszych budynków.
- hasSecurity i hasStorageRoom: cechy rzadziej występujące, ale z większymi wahaniami między miastami

#### 4. Różnice miejskie sugerują inny „profil budynków” i segmentację rynku
- Miasta różnią się nie tylko cenami, ale też strukturą cech: w jednych relatywnie częściej występują windy, w innych komórki lokatorskie, a gdzie indziej miejsca parkingowe.
- To wskazuje, że `city` jest zmienną silnie determinującą (nie tylko jako lokalizacja, ale też jako pośrednia informacja o typie zabudowy i standardu), a cechy `has*` mogą działać jako cechy doprecyzowujące segment wewnątrz miasta.

In [ ]:
plt.figure(figsize=(14, 8))

order_cities = df_sell.groupby('city')['price_per_m2'].median().sort_values(ascending=False).index

sns.boxplot(x='city', y='price_per_m2', data=df_sell, order=order_cities, hue='city', palette="viridis", legend=False)
plt.title('Rozkład Ceny za m² w poszczególnych miastach')
plt.xlabel('Miasto')
plt.ylabel('Cena za m² (PLN)')
plt.xticks(rotation=45)
plt.show()

### Wnioski z wykresu rozkładu ceny za m² w poszczególnych miastach
#### 1. Wyraźne różnice poziomu cen między miastami
- Wykres pokazuje silny efekt lokalizacji (city effect): mediany cen za m² znacząco się różnią w zależności od miasta.
- Warszawa ma najwyższą medianę i ogólnie najwyższy poziom cen za m² w całym zestawieniu. W kolejnej grupie znajdują się miasta typu Kraków oraz miasta Trójmiasta (Gdańsk / Gdynia), a następnie m.in. Wrocław.
- Na drugim biegunie znajdują się miasta o najniższych medianach (np. Radom, Częstochowa), co wskazuje na zupełnie inną skalę rynkową niż w największych aglomeracjach.

#### 2. Zmienność wewnątrz miast (IQR) sugeruje różną heterogeniczność rynku
- Szerokość pudełek (IQR) jest w wielu miastach wyraźna, co oznacza, że nawet w obrębie jednego miasta występuje istotne zróżnicowanie cen za m².
- Największe rynki (np. Warszawa, Kraków, Trójmiasto) zwykle mają większą rozpiętość rozkładu, co jest spójne z większą segmentacją: centrum vs peryferia, rynek premium vs standard, nowe inwestycje vs starsza zabudowa.

#### 3. Liczne wartości odstające – obecność segmentu premium i długi ogon rozkładu
- We wszystkich miastach widoczne są liczne outliery po stronie wysokich cen, szczególnie w największych ośrodkach. To sygnał, że rozkład jest prawoskośny (długi ogon w górę).
- Najwięcej i najwyższe wartości odstające występują w miastach o najwyższych medianach, co sugeruje dobrze rozwinięty segment mieszkań premium (np. apartamenty w najlepszych lokalizacjach, nowe inwestycje, oferta pod inwestora).

In [ ]:
def _ppm2(df):
    df = df.copy()
    if "price_per_m2" not in df.columns:
        df["price_per_m2"] = df["price"] / df["squareMeters"]
    return df

def _gdf(df):
    df = df.dropna(subset=["latitude", "longitude", "price_per_m2"]).copy()
    return gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326").to_crs("EPSG:3857")

def _waw(df):
    return df[df["city"].astype(str).str.strip().str.lower().eq("warszawa")].copy()

def _hex(ax, gdf, title, gridsize, mincnt):
    hb = ax.hexbin(gdf.geometry.x, gdf.geometry.y, C=gdf["price_per_m2"], reduce_C_function=np.median, gridsize=gridsize, mincnt=mincnt)
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    xmin, ymin, xmax, ymax = gdf.total_bounds
    mx, my = (xmax - xmin) * 0.05, (ymax - ymin) * 0.05
    ax.set_xlim(xmin - mx, xmax + mx); ax.set_ylim(ymin - my, ymax + my)
    ax.set_axis_off(); ax.set_title(title, pad=10)
    return hb

sell, rent = _ppm2(df_sell), _ppm2(df_rent)
g_sell_pl, g_rent_pl = _gdf(sell), _gdf(rent)
g_sell_waw, g_rent_waw = _gdf(_waw(sell)), _gdf(_waw(rent))

fig, ax = plt.subplots(1, 2, figsize=(16, 7))
hb1 = _hex(ax[0], g_sell_pl, "SELL (Polska): mediana price_per_m2 (min 10 / heks)", 60, 10)
hb2 = _hex(ax[1], g_rent_pl, "RENT (Polska): mediana price_per_m2 (min 10 / heks)", 60, 10)
fig.colorbar(hb1, ax=ax[0], fraction=0.046, pad=0.04, label="mediana price_per_m2")
fig.colorbar(hb2, ax=ax[1], fraction=0.046, pad=0.04, label="mediana price_per_m2")
plt.tight_layout(); plt.show()

fig, ax = plt.subplots(1, 2, figsize=(16, 7))
hb1 = _hex(ax[0], g_sell_waw, "SELL (Warszawa): mediana price_per_m2 (min 10 / heks)", 45, 10)
hb2 = _hex(ax[1], g_rent_waw, "RENT (Warszawa): mediana price_per_m2 (min 10 / heks)", 45, 10)
fig.colorbar(hb1, ax=ax[0], fraction=0.046, pad=0.04, label="mediana price_per_m2")
fig.colorbar(hb2, ax=ax[1], fraction=0.046, pad=0.04, label="mediana price_per_m2")
plt.tight_layout(); plt.show()

### Wnioski z map heksowych price_per_m2 

#### 1. Obraz dla Polski 
- Na mapach ogólnopolskich widać wyraźną polaryzację cen pomiędzy największymi rynkami a pozostałymi miastami. Dla sprzedaży  najwyższe wartości price_per_m2 koncentrują się w największych aglomeracjach, co jest spójne z intuicją rynkową: silny popyt, wyższe dochody oraz większy udział nowej zabudowy i atrakcyjnych lokalizacji.
- Dla wynajmu  rozkład jest podobny (najdroższe ośrodki nadal dominują), ale skala wartości jest oczywiście inna. W praktyce widać, że zarówno w sprzedaży, jak i w najmie Warszawa dominuje.
- Jednocześnie mapy potwierdzają, że analiza w skali kraju powinna być prowadzona z uwzględnieniem faktu, że porównujemy różne rynki lokalne, a nie jednorodny.

#### 2. Obraz dla Warszawy 
- W Warszawie na mapie sprzedaży widać mocny gradient cenowy: najwyższe wartości price_per_m2 układają się w centralnej części miasta i w wybranych kierunkach tworzą spójne „pasma” podwyższonych cen, podczas gdy peryferia (oraz obszary o mniejszej liczbie ofert) częściej wykazują niższe mediany.
- Na mapie wynajmu  wzorzec jest podobny w sensie geograficznym (również wyróżnia się rdzeń i obszary o wyższych stawkach), jednak struktura bywa bardziej rozproszona, co może wynikać z większej heterogeniczności ofert najmu (standard, metraż, segment rynku, krótkoterminowe ogłoszenia itp.).
- Co istotne, oba wykresy  sugerują, że w Warszawie ceny nie rosną równomiernie „od centrum w kółko”, tylko tworzą układ, który może być powiązany z osiami dobrej dostępności komunikacyjnej.

#### 3. Droższe lokalizacje wzdłuż M1
- Na mapach Warszawy da się zauważyć układ podwyższonych cen, który luźno przypomina liniowy korytarz. Jedną z naturalnych interpretacji jest wpływ infrastruktury transportowej – w szczególności pierwszej linii metra (M1), która łączy północ–południe i przebiega przez kluczowe obszary miejskie.

In [ ]:
GRIDSIZE_WAW, MINCNT = 80, 1
FEATURES = ["hasBalcony", "hasElevator", "hasSecurity", "hasParkingSpace", "hasStorageRoom"]

_waw = lambda df: df[df["city"].astype(str).str.strip().str.lower().eq("warszawa")].copy()
_gdf = lambda df: gpd.GeoDataFrame(df.dropna(subset=["latitude","longitude"]).copy(),
                                  geometry=gpd.points_from_xy(df.longitude, df.latitude),
                                  crs="EPSG:4326").to_crs("EPSG:3857")

def _hex(ax, gdf, title):
    hb = ax.hexbin(gdf.geometry.x, gdf.geometry.y, gridsize=GRIDSIZE_WAW, mincnt=MINCNT)
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    xmin, ymin, xmax, ymax = gdf.total_bounds
    mx, my = (xmax-xmin)*0.05, (ymax-ymin)*0.05
    ax.set_xlim(xmin-mx, xmax+mx); ax.set_ylim(ymin-my, ymax+my)
    ax.set_axis_off(); ax.set_title(title, pad=10)
    return hb

df = _waw(pd.concat([df_sell, df_rent], ignore_index=True))
has_cols = [c for c in df.columns if c.startswith("has")]
for c in has_cols: df[c] = df[c].where(df[c].isin(["yes","no"]))
features = [c for c in (FEATURES or has_cols) if c in df.columns]
gdf = _gdf(df)

ncols, nrows = 2, int(np.ceil(len(features)/2))
fig, axes = plt.subplots(nrows, ncols, figsize=(16, max(6, nrows*5)))
axes = np.array(axes).reshape(-1)

for i, feat in enumerate(features):
    sub = gdf[gdf[feat].eq("yes")]
    hb = _hex(axes[i], sub, f"Warszawa (SELL+RENT) | {feat}=yes: intensywność występowania (hexbin count)")
    fig.colorbar(hb, ax=axes[i], fraction=0.046, pad=0.04, label="liczba ofert w heksie")

for j in range(i+1, len(axes)): axes[j].axis("off")
plt.tight_layout(); plt.show()

### Wnioski z map intensywności występowania cech has* w Warszawie

#### 1. Co dokładnie pokazują mapy
- Mapy przedstawiają zagęszczenie ofert (hexbin count) spełniających warunek `hasX = yes`, czyli gdzie dana cecha (np. balkon, winda, ochrona) występuje w ogłoszeniach.

#### 2. Balkon (hasBalcony)
- Balkon jest cechą szeroko rozpowszechnioną – mapa jest stosunkowo „pełna”, co sugeruje, że w wielu częściach Warszawy oferty z balkonem pojawiają się regularnie.
- Największe natężenia występują w obszarach o wysokiej aktywności rynku (duża liczba ogłoszeń), co jest spójne z tym, że balkon jest standardem oferty w wielu segmentach

#### 3. Winda (hasElevator)
- hasElevator generuje wyraźnie silniejsze ogniska koncentracji niż balkon, co wskazuje na duży wolumen ofert w zabudowie, gdzie winda jest typowym wyposażeniem.
- Przestrzennie rozkład jest zgodny z intuicją urbanistyczną: winda częściej pojawia się tam, gdzie dominuje zabudowa wielopiętrowa lub nowsze inwestycje mieszkaniowe. Jednocześnie brak widocznych „punktowych” koncentracji sugeruje, że to cecha strukturalna dla typów budynków, a nie atrybut związany z pojedynczymi mikro-lokalizacjami.

#### 4. Ochrona (hasSecurity)
- hasSecurity jest cechą zdecydowanie rzadszą – mapa zawiera mniej silnych koncentracji, a wiele heksów ma niską intensywność.
- W praktyce oznacza to, że ochrona w ogłoszeniach jest bardziej charakterystyczna dla wybranych inwestycji/kompleksów (np. osiedla zamknięte, segment premium), a nie dla całego rynku.

#### 5. Miejsce parkingowe (hasParkingSpace)
- Cecha występuje częściej niż ochrona, ale jej natężenia są bardziej selektywne niż w przypadku balkonu.
- Przestrzennie może to odzwierciedlać większy udział inwestycji, w których parking jest przypisany do lokalu (garaże podziemne, miejsca postojowe), co bywa częstsze w nowszej zabudowie i określonych typach osiedli.

#### 6. Komórka lokatorska (hasStorageRoom)
- hasStorageRoom wykazuje stosunkowo szeroki zasięg, ale intensywności są wyraźnie niższe niż przy windzie.
- To sugeruje, że komórka lokatorska pojawia się jako cecha istotna, lecz mniej „domyślna” niż balkon czy winda – może być mocniej związana z konkretnymi typami budynków i standardem inwestycji.

In [ ]:
GRIDSIZE_WAW, MINCNT, TOP_Q, N_SEGMENTS, SMOOTH_WINDOW = 80, 10, 0.85, 35, 5

_ppm2 = lambda df: df.assign(price_per_m2=df["price"]/df["squareMeters"]) if "price_per_m2" not in df.columns else df.copy()
_waw  = lambda df: df[df["city"].astype(str).str.strip().str.lower().eq("warszawa")].copy()
_gdf  = lambda df: gpd.GeoDataFrame(df.dropna(subset=["latitude","longitude","price_per_m2"]).copy(),
                                   geometry=gpd.points_from_xy(df.longitude, df.latitude),
                                   crs="EPSG:4326").to_crs("EPSG:3857")

def _path(centers, n=N_SEGMENTS, w=SMOOTH_WINDOW):
    m = centers.mean(axis=0); X = centers - m
    v = np.linalg.eigh(np.cov(X.T))[1][:, -1]; v = v / np.linalg.norm(v)
    t = X @ v; edges = np.linspace(t.min(), t.max(), n + 1)
    pts = np.array([centers[(t>=a) & (t<b)].mean(axis=0) for a,b in zip(edges[:-1], edges[1:]) if ((t>=a) & (t<b)).any()])
    if w > 1 and len(pts) >= 3:
        k, p = w, w//2
        pad = np.pad(pts, ((p,p),(0,0)), mode="edge")
        pts = np.array([pad[i:i+k].mean(axis=0) for i in range(len(pts))])
    return pts

def plot_line_guess(df, label):
    g = _gdf(_waw(_ppm2(df)))
    fig, ax = plt.subplots(figsize=(10, 9))
    hb = ax.hexbin(g.geometry.x, g.geometry.y, C=g["price_per_m2"], reduce_C_function=np.median, gridsize=GRIDSIZE_WAW, mincnt=MINCNT)
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    xmin, ymin, xmax, ymax = g.total_bounds
    mx, my = (xmax-xmin)*0.05, (ymax-ymin)*0.05
    ax.set_xlim(xmin-mx, xmax+mx); ax.set_ylim(ymin-my, ymax+my); ax.set_axis_off()

    centers = hb.get_offsets()
    vals = np.asarray(hb.get_array())
    if hasattr(vals, "filled"): vals = vals.filled(np.nan)

    top = centers[vals >= np.nanquantile(vals, TOP_Q)]
    pts = _path(top)

    ax.scatter(top[:,0], top[:,1], s=10, alpha=0.9)
    if len(pts) >= 2: ax.plot(pts[:,0], pts[:,1], linewidth=3)

    ax.set_title(f"{label} | Warszawa: przybliżony przebieg osi najwyższych median price_per_m2 (top {(1-TOP_Q)*100:.0f}%)", pad=12)
    plt.colorbar(hb, ax=ax, fraction=0.046, pad=0.04, label="mediana price_per_m2 (PLN/m²)")
    plt.show()

plot_line_guess(df_sell, "SELL")
plot_line_guess(df_rent, "RENT")

### Wnioski z próby odtworzenia przebiegu linii metra na podstawie cen w Warszawie

#### 1. Cel i założenie eksperymentu
- W tej części analizy podjęto próbę przewidzenia przybliżonego przebiegu pierwszej linii metra (M1) w Warszawie, korzystając wyłącznie z informacji zawartych w danych ofertowych.
- Założeniem było, że dostęp do metra istotnie wpływa na atrakcyjność lokalizacji, a więc może przekładać się na wyższe średnie/medianowe ceny za m² w obszarach położonych wzdłuż osi transportowej.

#### 2. Metoda (intuicja)
- Na siatce heksów obliczono medianę price_per_m2 i wybrano obszary o najwyższych wartościach (top 15% heksów).
- Następnie na podstawie geometrii tych obszarów wyznaczono ciągłą linię, która ma reprezentować dominującą oś przestrzenną wysokich cen.
- Kluczowe jest to, że procedura nie korzystała z żadnych danych o transporcie (brak lokalizacji stacji, przebiegu torów, przystanków), więc wynik jest wyłącznie wnioskowaniem pośrednim na podstawie rozkładu cen.

#### 3. Wynik: zgodność z rzeczywistym przebiegiem M1
- Otrzymana linia jest mocno zbliżona do rzeczywistego przebiegu M1, zwłaszcza w wariancie dla sprzedaży (SELL). Widoczna jest dominująca orientacja północ–południe i przebieg przez obszary o najwyższych medianach cen.
- Dla wynajmu (RENT) zgodność również jest widoczna, ale rezultat jest mniej jednoznaczny i bardziej podatny na lokalne odchylenia

#### 4. Odchylenie na południu (kierunek Wilanowa)
- W obu wariantach zauważalne jest, że dolny fragment wyznaczonej trasy odchyla się bardziej w stronę Wilanowa niż rzeczywista M1.
- To prawdopodobnie efekt tego, że Wilanów jest obszarem o relatywnie wysokich cenach, a metoda oparta na cenie/m² traktuje takie dzielnice jako “silny sygnał”, mimo że nie wynika on bezpośrednio z przebiegu M1.
- Ten element pokazuje ograniczenie podejścia: rozkład cen odzwierciedla jednocześnie wiele czynników (centrum, standard zabudowy, prestiż lokalizacji, infrastruktura), a nie wyłącznie dostępność metra.

## 5. Statystyki liczbowe

In [ ]:
PCTS = [0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]

def ensure_price_per_m2(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "price_per_m2" not in df.columns and {"price", "squareMeters"}.issubset(df.columns):
        df["price_per_m2"] = df["price"] / df["squareMeters"]
    return df

def stats_table(df: pd.DataFrame, label: str) -> pd.DataFrame:
    df = ensure_price_per_m2(df)

    wanted = ["price", "squareMeters", "rooms", "centreDistance", "buildYear", "price_per_m2"]
    cols = [c for c in wanted if c in df.columns]

    desc = df[cols].describe(percentiles=PCTS).T  
    desc.insert(0, "dataset", label)

    return desc.round(2)

stats_sell = stats_table(df_sell, "SELL")
stats_rent = stats_table(df_rent, "RENT")

display(stats_sell)
display(stats_rent)

### Wnioski z statystyk opisowych

#### 1. Skala zbioru i kompletność danych
- Zbiór sprzedaży jest znacznie większy: 195 568 ofert vs 70 847 ofert dla wynajmu.
- buildYear ma braki: dostępny dla ok. 163 352/195 568 (~83,5%) w SELL oraz 51 165/70 847 (~72,2%) w RENT. W dalszej analizie/modelowaniu trzeba to uwzględnić

#### 2. Ceny – poziom i rozkład
SELL (cena całkowita):
- Mediana ceny to ok. 699 tys. PLN, a 75% ofert mieści się do 930 tys. PLN.
- Rozkład jest wyraźnie prawoskośny (długi ogon): 95% ofert jest poniżej ~1,59 mln PLN, ale maksimum sięga 3,25 mln PLN – widać segment premium/outliery.

RENT (czynsz miesięczny):
- Mediana czynszu to ok. 3 100 PLN, a 75% ofert do 4 490 PLN.
- Również silna prawoskośność: 95% poniżej 8 500 PLN, a maksimum 23 000 PLN wskazuje na segment premium / potencjalne obserwacje odstające.

#### 3. Cena za m² (price_per_m2) – kluczowa metryka porównawcza
- SELL: mediana price_per_m2 ≈ 12 979 PLN/m², a 90 percentyl ≈ 20 388 PLN/m². Zakres (min–max) jest szeroki (3 000 → 32 097 PLN/m²), co sugeruje duże zróżnicowanie lokalizacji i standardu.
- RENT: mediana price_per_m2 ≈ 65,96 PLN/m² (miesięcznie), 90 percentyl ≈ 99,16 PLN/m². Maksimum 189,47 PLN/m² to również sygnał segmentu premium.

#### 4. Metraż i liczba pokoi – „typowa” oferta
- Metraże są zbliżone, ale SELL jest minimalnie większy:
- SELL: mediana 54,6 m², 75% do 68,6 m²
- RENT: mediana 50 m², 75% do 64 m²
- Struktura pokoi:
- SELL: mediana 3 pokoje (25–75%: 2–3)
- RENT: mediana 2 pokoje (25–75%: 2–3)
To pasuje do intuicji: najem częściej dotyczy mniejszych lokali.

#### 5. Odległość od centrum (centreDistance)
- RENT jest przeciętnie bliżej centrum: średnia 3,86 km vs 4,35 km w SELL; także mediana jest niższa (3,38 km vs 3,98 km).
- To wspiera tezę, że rynek najmu jest bardziej skoncentrowany w lokalizacjach centralnych/okołocentralnych, gdzie popyt najemców jest najwyższy.

#### 6. Rok budowy (buildYear) – różnice między rynkami
- RENT ma wyraźnie młodszy zasób w danych: mediana 2001 vs 1994 w SELL.
- W RENT 75 percentyl to 2020, podczas gdy w SELL 2016. Może to wynikać z większej reprezentacji nowej zabudowy w ofertach najmu (np. inwestycje kupowane pod wynajem).

### Analiza Rozkładu Zmiennej Celu (Skośność)

Modele regresji liniowej najlepiej działają, gdy zmienna celu ma rozkład zbliżony do normalnego (Krzywa Gaussa).
Poniżej porównujemy rozkład surowej ceny (`price`) oraz jej logarytmu (`log_price`).

* **Skośność (Skewness):** Miara asymetrii rozkładu.
    * Wartość > 1 oznacza silną asymetrię prawostronną (dużo tanich, mało drogich).
    * Wartość bliska 0 oznacza rozkład symetryczny (idealny dla modelu).

In [ ]:
from scipy.stats import norm

def analyze_price_skewness(df, dataset_name, target_col='price'):
    # Wykonujemy kopie
    df_dist = df.copy()

    # Obliczenie logarytmu
    # log1p to log(x + 1) - bezpieczne dla wartości bliskich 0
    df_dist['log_price'] = np.log1p(df_dist[target_col])

    # Obliczenie skośności (Skewness)
    skew_raw = df_dist[target_col].skew()
    skew_log = df_dist['log_price'].skew()

    # Wizualizacja "Przed i Po"
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Rozkład surowy (PLN)
    sns.histplot(df_dist[target_col], bins=50, kde=True, color='skyblue', ax=axes[0])
    axes[0].set_title(f'Rozkład Ceny (Surowy) - {dataset_name}\nSkośność: {skew_raw:.2f} (Prawoskośny)', fontsize=12)
    axes[0].set_xlabel('Cena (PLN)')
    axes[0].axvline(df_dist[target_col].mean(), color='red', linestyle='--', label='Średnia')
    axes[0].legend()

    # Rozkład logarytmiczny
    sns.histplot(df_dist['log_price'], bins=50, kde=True, color='purple', ax=axes[1])
    
    # Dopasowanie idealnej krzywej normalnej dla porównania
    xmin, xmax = axes[1].get_xlim()
    x = np.linspace(xmin, xmax, 100)
    mu, std = df_dist['log_price'].mean(), df_dist['log_price'].std()
    p = norm.pdf(x, mu, std)
    
    # Skalowanie krzywej normalnej do wysokości histogramu
    axes[1].plot(x, p * len(df_dist) * (xmax - xmin)/50, 'k', linewidth=2, label='Rozkład Normalny')

    axes[1].set_title(f'Rozkład Logarytmu Ceny - {dataset_name}\nSkośność: {skew_log:.2f} (Zbliżona do zera)', fontsize=12)
    axes[1].set_xlabel('Log(Cena)')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

analyze_price_skewness(df_sell, "SPRZEDAŻ")
analyze_price_skewness(df_rent, "WYNAJEM")

### Wnioski z analizy rozkładu zmiennej celu:

Analiza rozkładu cen dla obu rynków (sprzedaż i wynajem) wykazała **silną prawoskośność (skośność > 1)**. Oznacza to, że choć większość ofert mieści się w niższych przedziałach cenowych, istnieje długi "ogon" bardzo drogich nieruchomości luksusowych, który drastycznie zaburza średnią i może negatywnie wpływać na stabilność modeli.

Wykresy jednoznacznie pokazują, że zastosowanie transformacji logarytmicznej (`np.log1p`) skutecznie normalizuje dane – rozkład "fioletowy" przypomina dzwon Gaussa, a jego skośność spada do poziomu bliskiego 0. Jest to ostateczne potwierdzenie, że **model powinien przewidywać `log_price`**, a nie surową kwotę, co jest kluczowe szczególnie dla poprawności działania modeli statystycznych.

## 6. Badanie Zależności i Korelacje

W tym kroku identyfikujemy zmienne, które mają największy wpływ na cenę mieszkania. Analiza obejmuje:
1.  **Analizę `log_price`:** Weryfikację, czy zlogarytmowanie ceny (zmienna celu) zwiększa siłę korelacji (co jest typowe dla rozkładów prawoskośnych).
2.  **Macierz korelacji:** Sprawdzenie siły zależności liniowej (Pearson) między cechami numerycznymi a zmienną celu.
3.  **Wizualizację trendów:** Wykresy punktowe badające relację Cena vs Metraż (dla kluczowych miast) oraz wpływ lokalizacji na cenę za m².

In [ ]:
def analyze_correlations(df, dataset_name, method='pearson'):
    df_corr = df.copy()
    df_corr['log_price'] = np.log1p(df_corr['price'])
    
    # Wybór zmiennych numerycznych do analizy
    target_cols = ['price', 'log_price', 'price_per_m2']
    feature_cols = [
        'squareMeters', 'rooms', 'floor', 'floorCount', 'buildYear', 
        'centreDistance', 'poiCount', 'schoolDistance', 'clinicDistance', 
        'restaurantDistance', 'kindergartenDistance'
    ]
    available_cols = target_cols + [c for c in feature_cols if c in df_corr.columns]
    
    # Obliczenie macierzy korelacji
    corr_matrix = df_corr[available_cols].corr(method=method)
    
    # Wizualizacja - Heatmapa
    plt.figure(figsize=(12, 8))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) # Ukrywamy górny trójkąt (duplikaty)
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', 
                mask=mask, vmin=-1, vmax=1, center=0, cbar_kws={"shrink": .8})
    plt.title(f'Macierz Korelacji ({dataset_name})')
    plt.show()
    
    # Ranking najważniejszych cech (Tabela)
    print(f"\nTOP 10 Korelacji ze zmiennymi celu ({dataset_name}):")
    
    # Pobieramy korelacje dla log_price i price_per_m2
    rank_log = corr_matrix['log_price'].drop(target_cols, errors='ignore').abs().sort_values(ascending=False).head(10)
    rank_m2 = corr_matrix['price_per_m2'].drop(target_cols, errors='ignore').abs().sort_values(ascending=False).head(10)
    
    # Funkcje pomocnicze do budowy tabeli
    idx1, val1 = rank_log.index.tolist(), corr_matrix.loc[rank_log.index, 'log_price'].values
    idx2, val2 = rank_m2.index.tolist(), corr_matrix.loc[rank_m2.index, 'price_per_m2'].values
    max_len = max(len(idx1), len(idx2))
    
    def pad(l, size, fill): return l + [fill] * (size - len(l))

    ranking_df = pd.DataFrame({
        'Cecha (log_price)': pad(idx1, max_len, '-'),
        'Korelacja (log_price)': pad(list(val1), max_len, np.nan),
        ' | ': ['|'] * max_len,
        'Cecha (price_per_m2)': pad(idx2, max_len, '-'),
        'Korelacja (price_per_m2)': pad(list(val2), max_len, np.nan)
    })
    
    # Wyświetlenie sformatowanej tabeli
    display(ranking_df.style.background_gradient(
        cmap='coolwarm', 
        subset=['Korelacja (log_price)', 'Korelacja (price_per_m2)'], 
        vmin=-1, vmax=1
    ).format("{:.3f}", subset=['Korelacja (log_price)', 'Korelacja (price_per_m2)']))

# Wywołanie dla wyczyszczonych danych
analyze_correlations(df_sell, "SPRZEDAŻ")
analyze_correlations(df_rent, "WYNAJEM")

### Wnioski z analizy korelacji:

Na podstawie macierzy korelacji dla obu podzbiorów (sprzedaży i wynajmu) można sformułować następujące spostrzeżenia dotyczące czynników cenotwórczych:

1.  **Fundamentalna rola metrażu (Różnice rynkowe):**
    Najsilniejszym predyktorem ceny całkowitej jest powierzchnia (`squareMeters`). Co ciekawe, korelacja ta jest **silniejsza dla rynku wynajmu (0.744)** niż dla sprzedaży (0.608). Sugeruje to, że ceny najmu są bardziej "sztywno" powiązane z wielkością lokalu, podczas gdy na cenę sprzedaży większy wpływ mogą mieć inne czynniki (np. standard wykończenia, prestiż dzielnicy, stan prawny), których proste cechy numeryczne nie w pełni oddają.

2.  **Lokalizacja - "Usługi ważniejsze niż Centrum":**
    W przypadku ceny za m² (`price_per_m2`), silniejszą korelację niż sama odległość od centrum (`centreDistance`) wykazuje liczba punktów użyteczności publicznej (`poiCount`: ~0.23) oraz bliskość restauracji (`restaurantDistance`: ~-0.18).
    * **Wniosek:** Dla wartości nieruchomości ważniejsza od geometrycznego środka miasta jest **funkcjonalność okolicy** (dostęp do usług, gastronomii). Potwierdza to koncepcję miasta policentrycznego.

3.  **Wiek budynku (Kupno vs Wynajem):**
    Zmienna `buildYear` odgrywa znacznie większą rolę w sprzedaży (korelacja 0.204 z ceną za m²) niż w wynajmie (tylko 0.087).
    * **Interpretacja:** Kupujący traktują mieszkanie jako inwestycję długoterminową, więc nowsze budownictwo jest wyceniane znacznie wyżej (mniejsze ryzyko remontów, wyższa efektywność energetyczna). Dla najemcy rok budowy jest drugorzędny – liczy się obecny standard i lokalizacja.

4.  **Efekt "Hurtu" (Korelacja ujemna):**
    W danych sprzedażowych widoczna jest ujemna korelacja między metrażem a ceną za m² (`-0.103`). Jest to klasyczne zjawisko rynkowe: mniejsze lokale (kawalerki) mają najwyższą cenę jednostkową, a wraz ze wzrostem metrażu cena za m² spada. W danych najmu ta zależność zanika lub jest lekko dodatnia, co sugeruje inną dynamikę popytu na duże mieszkania na wynajem.

5.  **Uzasadnienie dla transformacji logarytmicznej:**
    Analiza potwierdza, że zmienna `log_price` wykazuje wyższe wartości korelacji z cechami (np. 0.608) niż surowa cena. Potwierdza to zasadność trenowania modeli na logarytmie ceny w celu zlinearyzowania zależności wykładniczych.

6.  **Problem współliniowości:**
    Bardzo wysoka korelacja między `squareMeters` a `rooms` (często > 0.8) wskazuje na redundancję tych cech. W modelach wrażliwych na współliniowość (np. Regresja Liniowa) może to destabilizować wagi cech.

### Analiza porównawcza rynków lokalnych

Rynek nieruchomości w Polsce nie jest jednorodny. Aby zweryfikować hipotezę, że cena mieszkania zależy nie tylko od jego metrażu, ale także od miasta, w którym się znajduje, przeprowadzamy analizę dla 4 największych rynków: Warszawy, Krakowa, Wrocławia i Poznania.

Wykorzystujemy wykres punktowy z naniesioną linią regresji (`lmplot`), rozbity na panele. Pozwoli to porównać nachylenie krzywych cenowych – im bardziej stroma linia, tym droższy jest każdy kolejny metr kwadratowy w danym mieście.

In [ ]:
top_cities_keys = ['warszawa', 'krakow', 'wroclaw', 'poznan'] 

# Słownik do mapowania nazw na ładne etykiety z polskimi znakami
city_labels = {
    'warszawa': 'Warszawa', 
    'krakow': 'Kraków', 
    'wroclaw': 'Wrocław', 
    'poznan': 'Poznań'
}

# Tworzymy kopię danych tylko dla danych miast
df_plot = df_sell[df_sell['city'].isin(top_cities_keys)].copy()
df_plot['city_label'] = df_plot['city'].map(city_labels)

g = sns.lmplot(
    data=df_plot, 
    x='squareMeters', 
    y='price', 
    col='city_label',   
    col_wrap=2,         
    hue='city_label',   
    height=4, 
    aspect=1.5,
    scatter_kws={'alpha': 0.3, 's': 15}, 
    line_kws={'color': '#333333'}      
)
g.figure.suptitle('Zależność Ceny całkowitej od Metrażu w największych miastach', y=1.02, fontsize=16)
plt.show()

### Wnioski z analizy trendów w miastach (Cena vs Metraż):

1.  **Potwierdzenie liniowości relacji:**
    We wszystkich analizowanych aglomeracjach (Warszawa, Kraków, Wrocław, Poznań) obserwujemy silną, dodatnią korelację liniową. Punkty układają się w zwarte "chmury" wzdłuż linii regresji.

2.  **Zróżnicowanie cen krańcowych (Nachylenie prostej):**
    Kluczową obserwacją jest różnica w kącie nachylenia linii trendu (współczynnik kierunkowy).
    * W **Warszawie** linia jest najbardziej stroma. Oznacza to, że każdy dodatkowy metr kwadratowy powierzchni podnosi cenę końcową znacznie bardziej niż w pozostałych miastach.
    * W **Poznaniu** linia jest bardziej płaska, co sugeruje mniejszą wrażliwość ceny całkowitej na wzrost metrażu.
    * **Implikacja dla modelu:** Model nie może traktować metrażu uniwersalnie. Konieczne jest uwzględnienie interakcji między zmiennymi, który "zrozumie", że 50 m² w Warszawie to inna klasa cenowa niż 50 m² we Wrocławiu.

3.  **Heteroskedastyczność (Zmienność wariancji):**
    Warto zauważyć, że im większy metraż, tym bardziej punkty są rozproszone wokół linii (chmura się rozszerza). Oznacza to, że ceny małych mieszkań są bardzo przewidywalne, natomiast w segmencie dużych apartamentów wariancja cen jest ogromna (zależą one mocniej od standardu wykończenia niż tylko od powierzchni).

### Analiza wpływu lokalizacji (Mapa gęstości)

Drugim kluczowym czynnikiem cenotwórczym jest odległość od centrum (`centreDistance`). Ponieważ zbiór danych jest duży, zwykły wykres punktowy byłby nieczytelny (problem nakładania się punktów).

Zamiast tego stosujemy wykres heksagonalny (hexbin plot), który działa jak mapa termiczna.
* **Oś X:** Odległość od centrum (km).
* **Oś Y:** Cena za m² (PLN).
* **Kolor:** Liczba ofert w danym obszarze (skala logarytmiczna).

Dzięki temu zobaczymy nie tylko trend cenowy, ale także strukturę podaży – w jakiej odległości od centrum dostępnych jest najwięcej mieszkań.

In [ ]:
plt.figure(figsize=(10, 6))

# Odsiewamy skrajne wartości (outliery) dla czytelności wykresu
df_hex = df_sell[
    (df_sell['centreDistance'] < 15) & 
    (df_sell['price_per_m2'] < 35000)
]

hb = plt.hexbin(
    df_hex['centreDistance'], 
    df_hex['price_per_m2'], 
    gridsize=40,    
    cmap='inferno', 
    mincnt=1,       
    bins='log'     
)

cb = plt.colorbar(hb, label='Liczba ofert (skala log)')
plt.title('Gęstość ofert: Cena za m² vs Odległość od centrum (Cała Polska)')
plt.xlabel('Odległość od centrum (km)')
plt.ylabel('Cena za m² (PLN)')
plt.grid(True, alpha=0.3)
plt.show()

### Wnioski z analizy wpływu lokalizacji (Cena za m² vs Odległość od centrum):

1.  **Nieliniowa dynamika spadku cen (Krzywa renty gruntowej):**
    Wykres ujawnia charakterystyczny kształt litery "L" (lub rozkładu hiperbolicznego $1/x$).
    * **Strefa Centrum (0-3 km):** Obserwujemy ekstremalnie wysokie ceny i ogromną wariancję (pionowy słupek po lewej stronie). Tutaj lokalizacja generuje "premię za centralność", która drastycznie winduje ceny luksusowych apartamentów.
    * **Strefa Miejska (3-10 km):** Następuje gwałtowny spadek cen, po czym wykres zaczyna się wypłaszczać.
    * **Peryferia (>10 km):** Ceny stabilizują się na stałym, niższym poziomie. Dalsze oddalanie się od centrum (np. z 12 na 15 km) ma już minimalny wpływ na spadek ceny.
    * **Implikacja dla modelu:** Zastosowanie prostej korelacji liniowej dla cechy `centreDistance` będzie błędem (niedoszacuje cen w centrum i przeszacuje na peryferiach). Lepiej sprawdzi się model nieliniowy, np. k-NN.

2.  **Koncentracja rynku (Analiza gęstości):**
    Najjaśniejsze obszary na mapie (żółte/pomarańczowe heksagony) wskazują na "masowy" segment rynku.
    * Największa podaż mieszkań występuje w pasie **3–8 km od centrum**.
    * Ceny w tym segmencie są relatywnie jednorodne i skupione wokół średniej rynkowej (np. 10–15 tys. PLN/m²). To tutaj modele predykcyjne powinny osiągać najwyższą skuteczność ze względu na dużą liczbę próbek uczących i mniejszy rozrzut cenowy niż w ścisłym centrum.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# --- 1. PRZYGOTOWANIE DANYCH ---
def get_preprocessor(numeric_features, categorical_features):
    """Tworzy i zwraca obiekt ColumnTransformer do przetwarzania danych."""
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )
    return preprocessor

def prepare_data(df, city_filter=None, exclude_city=None, target_col='price'):
    """Filtruje dane, wybiera cechy, logarytmuje cel i dzieli na train/test."""
    data = df.copy()
    
    # Filtrowanie miastaf
    if city_filter:
        data = data[data['city'] == city_filter]

    if exclude_city:
        data = data[data['city'] != exclude_city]
    
    # Wybór cech
    numeric_features = ['squareMeters', 'rooms', 'floor', 'buildYear', 'centreDistance', 'poiCount']
    categorical_features = ['city'] if city_filter is None else []
    
    # Bezpieczny wybór tylko istniejących kolumn
    numeric_features = [c for c in numeric_features if c in data.columns]
    
    X = data[numeric_features + categorical_features]
    y = data[target_col]
    
    # Logarytmowanie celu
    y_log = np.log1p(y)
    
    # Podział na zbiór treningowy i testowy
    X_train, X_test, y_train_log, y_test_log = train_test_split(
        X, y_log, test_size=0.2, random_state=42
    )
    
    return X_train, X_test, y_train_log, y_test_log, numeric_features, categorical_features

# --- 2. FUNKCJE MODELI ---
def train_linear_regression(X_train, y_train, preprocessor):
    """Trenuje model Regresji Liniowej."""
    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', LinearRegression())])
    model.fit(X_train, y_train)
    return model

def train_decision_tree(X_train, y_train, preprocessor):
    """Trenuje model Drzewa Decyzyjnego."""
    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', DecisionTreeRegressor(max_depth=10, random_state=42))])
    model.fit(X_train, y_train)
    return model

def train_knn(X_train, y_train, preprocessor):
    """Trenuje model k-Najbliższych Sąsiadów."""
    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', KNeighborsRegressor(n_neighbors=5))])
    model.fit(X_train, y_train)
    return model

# --- 3. EWALUACJA ---
def evaluate_model(model, X_test, y_test_log, model_name, ax=None):
    """Dokonuje predykcji, liczy metryki i opcjonalnie rysuje wykres."""
    # Predykcja
    y_pred_log = model.predict(X_test)
    
    # Odwrócenie logarytmu (powrót do PLN)
    y_test_true = np.expm1(y_test_log)
    y_pred_true = np.expm1(y_pred_log)
    
    # Metryki
    mae = mean_absolute_error(y_test_true, y_pred_true)
    rmse = np.sqrt(mean_squared_error(y_test_true, y_pred_true))
    r2 = r2_score(y_test_true, y_pred_true)
    
    # Wizualizacja
    if ax:
        sns.scatterplot(x=y_test_true, y=y_pred_true, alpha=0.3, ax=ax)
        max_val = max(y_test_true.max(), y_pred_true.max())
        ax.plot([0, max_val], [0, max_val], 'r--') # Idealna linia
        ax.set_title(f"{model_name}\nR² = {r2:.3f}")
        ax.set_xlabel("Rzeczywista Cena")
        ax.set_ylabel("Przewidywana Cena")
        ax.grid(True, alpha=0.3)
        
    return {
        'Model': model_name,
        'MAE (zł)': mae,
        'RMSE (zł)': rmse,
        'R²': r2
    }

# --- 4. GŁÓWNA FUNKCJA STERUJĄCA ---
def run_ml_experiments(df, title, city_filter=None, exclude_city=None):
    print(f"\n{'='*60}")
    print(f"URUCHAMIAM EKSPERYMENTY: {title}")
    if city_filter: print(f"Filtr miasta: {city_filter}")
    print(f"{'='*60}")
    
    # 1. Przygotowanie danych
    X_train, X_test, y_train, y_test, num_feats, cat_feats = prepare_data(df, city_filter, exclude_city)
    
    # 2. Przygotowanie preprocessora
    preprocessor = get_preprocessor(num_feats, cat_feats)
    
    # 3. Trening modeli (każdy osobną funkcją)
    models = [
        ('Regresja Liniowa', train_linear_regression(X_train, y_train, preprocessor)),
        ('Drzewo Decyzyjne', train_decision_tree(X_train, y_train, preprocessor)),
        ('k-NN (k=5)', train_knn(X_train, y_train, preprocessor))
    ]
    
    # 4. Ewaluacja i Raportowanie
    results = []
    plt.figure(figsize=(18, 5))
    
    for i, (name, model) in enumerate(models):
        ax = plt.subplot(1, 3, i+1)
        metrics = evaluate_model(model, X_test, y_test, name, ax)
        results.append(metrics)
        
    plt.tight_layout()
    plt.show()
    
    # Tabela wyników
    results_df = pd.DataFrame(results)
    display(results_df.style.highlight_max(axis=0, subset=['R²'], color='green')
                      .highlight_min(axis=0, subset=['MAE (zł)', 'RMSE (zł)'], color='green')
                      .format("{:.2f}", subset=['MAE (zł)', 'RMSE (zł)'])
                      .format("{:.3f}", subset=['R²']))

In [ ]:
run_ml_experiments(df_sell, "RYNEK SPRZEDAŻY (Cała Polska)")

run_ml_experiments(df_sell, "RYNEK SPRZEDAŻY (Tylko Warszawa)", city_filter="warszawa")

run_ml_experiments(df_sell, "RYNEK SPRZEDAŻY (Cała Polska bez Warszawy)", exclude_city="warszawa")

run_ml_experiments(df_rent, "RYNEK SPRZEDAŻY (Cała Polska)")

run_ml_experiments(df_rent, "RYNEK SPRZEDAŻY (Tylko Warszawa)", city_filter="warszawa")

run_ml_experiments(df_rent, "RYNEK SPRZEDAŻY (Cała Polska bez Warszawy)", exclude_city="warszawa")

## 7. Wnioski i interpretacja wyników modelowania

Na podstawie przeprowadzonych eksperymentów i analizy metryk błędów (MAE, RMSE, $R^2$), sformułowano następujące wnioski dotyczące skuteczności algorytmów w predykcji cen nieruchomości:

1.  **Dominacja algorytmu k-Najbliższych Sąsiadów (k-NN):**
    Najlepsze wyniki predykcyjne (najwyższy współczynnik $R^2$ oraz najniższe błędy MAE/RMSE) osiągnął model **k-NN**. Wynik ten ma silne uzasadnienie merytoryczne:
    * **Natura rynku:** Wycena nieruchomości w praktyce opiera się na tzw. *podejściu porównawczym* (analiza transakcji podobnych lokali w okolicy). Algorytm k-NN działa w sposób analogiczny – estymuje cenę na podstawie średniej z $k$ najbardziej zbliżonych punktów w wielowymiarowej przestrzeni cech.
    * **Lokalność:** Dzięki uwzględnieniu cech takich jak `city` oraz `centreDistance`, algorytm skutecznie znajduje sąsiadów o podobnej charakterystyce lokalizacyjnej, co jest kluczowe dla ceny.

2.  **Znaczenie skalowania danych:**
    Wysoka skuteczność k-NN potwierdza, że zastosowany **StandardScaler** został użyty poprawnie. Ponieważ k-NN opiera się na obliczaniu odległości euklidesowych, sprowadzenie metrażu (rzędu 50 m²) i odległości od centrum (rzędu 5 km) do wspólnej skali było krytyczne dla sukcesu tego modelu.

3.  **Ograniczenia Regresji Liniowej:**
    Model regresji liniowej osiągnął słabsze wyniki w porównaniu do k-NN. Wskazuje to, że zależności na rynku nieruchomości nie są w pełni liniowe. Przykładowo, wpływ odległości od centrum na cenę nie jest stały (cena spada gwałtownie blisko centrum i stabilizuje się na peryferiach), co dla prostej regresji jest trudne do odwzorowania bez zaawansowanej inżynierii cech.

4.  **Wydajność Drzewa Decyzyjnego:**
    Drzewo decyzyjne uplasowało się zazwyczaj pośrodku stawki (lub blisko k-NN). Choć dobrze radzi sobie z nieliniowością, może mieć tendencję do "schodkowania" predykcji (przypisywania tej samej ceny dla grupy mieszkań w jednym liściu), podczas gdy k-NN oferuje bardziej płynną interpolację cen, co w przypadku zmiennej ciągłej (ceny) daje mniejszy błąd średni.

**Podsumowując:** Eksperyment wykazał, że dla tego zbioru danych podejście oparte na podobieństwie (k-NN) jest skuteczniejsze niż podejście oparte na regułach (Drzewa) lub prostych równaniach liniowych (Regresja).